In [2]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777) # for reproducibility

In [4]:
"""
09-2에서는 layer1, 2 두개로 했다면s
이제는 좀 넓게 해보자.

실험을 위해 10개로 나가는?(근데 보통은 입력보다는 적게하긴 한다.)

layer나가면 10개를 입력받아서 1개의 y를 내도록..
이런걸 wide하다고 함.
------------------------------------
Layer를 깊게 더 여러개 할 수도 있겠죠.
"""
# XOR Data set
x_data = [[0, 0], [0, 1], [1, 0], [1, 1]]
y_data = [[0], [1], [1], [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

"""
# Hypothesis in Wide NN for XOR

# 첫번째 layer 구성
# 입력값은 2로 정해져있고, 출력값을 10으로 주면 된다. 원하는대로 임의대로
W1 = tf.Variable(tf.random_normal([2, 10]), name='weight1') 
b1 = tf.Variable(tf.random_normal([10]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)


# 그후 layer2의 in은 1의 out과 같게
# Layer를 뒤에 하나 더 두자. layer1의 output을 input으로 쓰자.
W2 = tf.Variable(tf.random_normal([10, 1]), name='weight2') 
b2 = tf.Variable(tf.random_normal([1]), name='bias2') 

# layer1의 값을 연결을 시킴
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2) 


"""

# 첫번째 layer 구성
# 입력값은 2로 정해져있고, 출력값을 10으로 주면 된다. 원하는대로 임의대로
W1 = tf.Variable(tf.random_normal([2, 10]), name='weight1') 
b1 = tf.Variable(tf.random_normal([10]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([10, 10]), name='weight2') 
b2 = tf.Variable(tf.random_normal([10]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([10, 10]), name='weight3') 
b3 = tf.Variable(tf.random_normal([10]), name='bias2')
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([10, 1]), name='weight4') 
b4 = tf.Variable(tf.random_normal([1]), name='bias2') 
hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4) 

# cost/loss function
# tf.log는 왜 하는거지???????
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
# cost를 최소화하기위해 gradientdescent 쓴다.
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Accuracy computation
# True if hypothesis > 0.5 else False
# 정확도 계산하기위한 것들
# hypothesis가 0.5보다 크면 True가되서 1이된다.
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
# 그 값이 실제로 갖고 있는 Y값과 같은지를 비교하고 
# 이것도 True, False로 나뉘어서 casting해서 True는 1로되서 그걸 더해서 평균내면 정확도가 나오죠.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                X: x_data, Y:y_data}), sess.run([W1, W2]))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], 
                        feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: \n", h, "\nCorrect: \n", c, "\nAccuracy: \n", a)
    # 큰차이는 없지만, Wide한 것은 기존의 것보다 작은 값은 더 작아졌고, 큰 값은 더커짐. 이말은 모델이 더 잘 학습됬다라는 걸 의미함.

0 2.05893 [array([[ 0.59931177, -0.1836307 , -1.16583848, -1.07160628,  1.24879658,
        -2.22273946,  1.66687775, -1.34460187,  2.32012892, -1.41912353],
       [ 0.82809466, -1.19920313, -0.95467597, -0.52923536,  0.02491293,
        -0.63629776,  0.71442437, -0.27890295,  1.5067935 , -0.68222672]], dtype=float32), array([[ 0.81926018,  0.05565166, -0.04198147, -2.41935492,  0.93914074,
        -0.4195708 ,  1.03976798,  0.58087099,  0.5616684 , -0.37637591],
       [ 0.13856336, -0.1140713 ,  0.40599686, -1.32757819, -0.68350017,
        -0.49460396, -1.54878283, -2.10489726, -0.22214317,  0.76305151],
       [ 0.2102358 ,  0.7777887 ,  1.41914272, -0.72000307, -0.20443802,
        -0.58258605, -0.29318377, -0.08519008, -0.58913922,  1.15785193],
       [-1.65778112, -0.40266535, -0.59629679,  1.22278404,  0.52946049,
         0.26992404, -0.75276321,  0.40055233,  0.34561399, -1.56398726],
       [ 0.3158699 ,  0.32232869, -0.85686815,  0.33438855,  0.46120355,
         0.889006

600 0.677782 [array([[ 0.59603369, -0.3036949 , -1.20022666, -1.07088542,  1.26419044,
        -2.22862053,  1.72290194, -1.33822525,  2.33014894, -1.39150465],
       [ 0.77210206, -1.26674616, -0.99802184, -0.49683359,  0.0185412 ,
        -0.66400027,  0.90228534, -0.23403998,  1.51790178, -0.55496776]], dtype=float32), array([[ 0.83135343,  0.04681024, -0.10322344, -2.35933781,  0.8662681 ,
        -0.56983238,  1.1131953 ,  0.61585271,  0.55560601, -0.35123175],
       [ 0.23911855, -0.13921125,  0.35366234, -1.35617578, -0.76591355,
        -0.55873775, -1.53305244, -2.06616759, -0.22306399,  0.79449856],
       [ 0.34185243,  0.76029605,  1.33073509, -0.75465918, -0.31955999,
        -0.65892446, -0.25695798, -0.02377059, -0.61819452,  1.17046285],
       [-1.54169881, -0.41640618, -0.69660079,  1.21399212,  0.40849602,
         0.16102947, -0.69199395,  0.46668351,  0.31179115, -1.55414736],
       [ 0.33166695,  0.30258355, -0.89186215,  0.37753111,  0.40999004,
         0.770

1200 0.60235 [array([[ 0.62223548, -0.85097563, -1.40684128, -1.14193213,  1.36527312,
        -2.27563405,  1.85419619, -1.3145951 ,  2.50153303, -1.43342507],
       [ 0.73332042, -1.59714997, -1.20783603, -0.67004436,  0.22447194,
        -0.83371252,  1.25320292, -0.1361682 ,  1.82931614, -0.61505449]], dtype=float32), array([[ 0.817101  ,  0.07703421, -0.17749684, -2.32293582,  0.92433137,
        -0.64493161,  1.2029562 ,  0.60410434,  0.58005977, -0.31599849],
       [ 0.29251122, -0.12869766,  0.33047768, -1.48260796, -0.83595955,
        -0.59051681, -1.58367133, -2.29311752, -0.2585766 ,  0.75455528],
       [ 0.39974999,  0.78134102,  1.27689803, -0.87579745, -0.37420633,
        -0.67672664, -0.29762921, -0.29747307, -0.71571499,  1.07631755],
       [-1.50216424, -0.38730988, -0.7850346 ,  1.16623116,  0.42110258,
         0.14183502, -0.67061114,  0.25771257,  0.19974698, -1.67501378],
       [ 0.32932442,  0.31754887, -0.93257433,  0.38872024,  0.44339091,
         0.695

1800 0.253592 [array([[ 0.79199082, -1.76137662, -1.82915556, -1.6893152 ,  1.57648051,
        -2.41350651,  2.04821062, -1.29859591,  2.86302471, -1.52251518],
       [ 0.99393284, -2.1590023 , -1.62335956, -1.69798374,  0.77561325,
        -1.26521671,  1.64875388,  0.04259157,  2.3754847 , -0.84510696]], dtype=float32), array([[  8.01518619e-01,   1.77549809e-01,  -2.08769083e-01,
         -2.54061890e+00,   1.00617695e+00,  -7.47092962e-01,
          1.25897014e+00,   7.10021734e-01,   9.84191954e-01,
          2.19069213e-01],
       [  2.80158579e-01,  -8.56072232e-02,   3.36180240e-01,
         -1.59785652e+00,  -9.42282557e-01,  -6.17977202e-01,
         -1.68645799e+00,  -2.68954349e+00,  -4.28063065e-01,
          6.31962597e-01],
       [  3.26731026e-01,   8.40163350e-01,   1.26796734e+00,
         -8.95722151e-01,  -4.30835634e-01,  -6.92479491e-01,
         -3.94809246e-01,  -6.92204475e-01,  -9.99207973e-01,
          7.98568130e-01],
       [ -1.78613067e+00,  -2.89620

2400 0.0354663 [array([[ 0.89821929, -2.17866683, -2.03960848, -2.09408069,  1.62644506,
        -2.50974607,  2.15243006, -1.29169679,  3.05266976, -1.55012238],
       [ 1.20503485, -2.49095035, -1.8737185 , -2.21736908,  1.02815688,
        -1.51311839,  1.83245993,  0.17256875,  2.65658784, -0.93977189]], dtype=float32), array([[ 0.91314054,  0.22025916, -0.19996724, -2.79870868,  1.04711461,
        -0.76419473,  1.26212609,  0.80389732,  1.31144571,  0.50488007],
       [ 0.33811685, -0.10172007,  0.37757877, -1.75095642, -1.05139732,
        -0.6113978 , -1.81665158, -2.93740416, -0.55691558,  0.81629574],
       [ 0.34620166,  0.81929022,  1.30754435, -0.97711426, -0.51435679,
        -0.67490923, -0.52308643, -0.92176908, -1.19760454,  0.90089917],
       [-2.02912188, -0.33956054, -0.79990608,  1.86101115,  0.66366243,
         0.24395709, -0.81946117, -0.06731265, -1.02163315, -2.95034504],
       [ 0.52747768,  0.45402357, -0.94322884, -0.25580516,  0.50565434,
         0.5

...

7600 0.00197728 [array([[ 1.035254  , -2.51416516, -2.23145914, -2.5075078 ,  1.68028045,
        -2.62522078,  2.22394419, -1.29280066,  3.21440244, -1.57475019],
       [ 1.34302962, -2.78712702, -2.11004949, -2.60177779,  1.22668421,
        -1.77485216,  1.95661843,  0.30893758,  2.89434338, -1.0081501 ]], dtype=float32), array([[ 1.03404689,  0.25091526, -0.19451495, -2.98049116,  1.08810806,
        -0.77299911,  1.30613995,  0.9022634 ,  1.57808602,  0.71230924],
       [ 0.43830597, -0.11416536,  0.421471  , -1.94744611, -1.15635204,
        -0.60587955, -1.92404699, -3.08223939, -0.65955091,  1.14669144],
       [ 0.41145369,  0.80175859,  1.34639406, -1.108199  , -0.5962016 ,
        -0.65857458, -0.61787158, -1.04334283, -1.33984303,  1.15286589],
       [-2.22698116, -0.41048619, -0.77632272,  2.19497299,  0.73363888,
         0.36356184, -0.85762835, -0.06969731, -1.59648526, -3.25762415],
       [ 0.68119913,  0.49524102, -0.93755621, -0.50387663,  0.53169858,
         0.

8200 0.00175927 [array([[ 1.04070663, -2.52530622, -2.23835254, -2.52142763,  1.68226051,
        -2.62985301,  2.22608805, -1.2930454 ,  3.21996403, -1.57544792],
       [ 1.34789991, -2.79688835, -2.11823487, -2.61472893,  1.23353493,
        -1.78439009,  1.96015775,  0.31372476,  2.90218639, -1.01004505]], dtype=float32), array([[ 1.03844953,  0.25192106, -0.19437708, -2.98679519,  1.08946395,
        -0.77345026,  1.30808532,  0.90521598,  1.5869658 ,  0.71879637],
       [ 0.44208494, -0.11444309,  0.42302138, -1.9540267 , -1.15959394,
        -0.60580802, -1.92727852, -3.08601594, -0.66272509,  1.16005945],
       [ 0.41399989,  0.80129766,  1.34772623, -1.11260223, -0.59869152,
        -0.65808868, -0.62059963, -1.0464164 , -1.34412301,  1.16335118],
       [-2.23415637, -0.4130801 , -0.77581239,  2.20704079,  0.73636901,
         0.36838305, -0.85766906, -0.06885142, -1.6150769 , -3.26692653],
       [ 0.68669653,  0.4965876 , -0.93734169, -0.51235622,  0.5325557 ,
         0.

8700 0.00160987 [array([[ 1.04484391, -2.53367805, -2.24355292, -2.53189421,  1.6837697 ,
        -2.63337183,  2.22769141, -1.29322422,  3.2241528 , -1.57597399],
       [ 1.35157645, -2.80421996, -2.12440515, -2.62447238,  1.23870468,
        -1.79159307,  1.96279347,  0.31733072,  2.90807748, -1.01145518]], dtype=float32), array([[ 1.04178047,  0.25267655, -0.1942752 , -2.99155593,  1.09047937,
        -0.77380246,  1.30956089,  0.90741676,  1.59363639,  0.72366095],
       [ 0.44494069, -0.11464468,  0.42419088, -1.95896673, -1.16201818,
        -0.60575008, -1.92969179, -3.08881617, -0.66509736,  1.170187  ],
       [ 0.41592583,  0.80095696,  1.34872437, -1.11590815, -0.6005528 ,
        -0.65772545, -0.62263417, -1.04869306, -1.34731996,  1.17130101],
       [-2.23958254, -0.41503838, -0.77544117,  2.21615267,  0.73843586,
         0.37204933, -0.85765558, -0.06818958, -1.62902749, -3.2739141 ],
       [ 0.69085121,  0.49759826, -0.93719268, -0.51874983,  0.53319585,
         0.

9300 0.00145972 [array([[ 1.04939866, -2.54282355, -2.24925852, -2.54332876,  1.68543899,
        -2.63724637,  2.22944283, -1.29343879,  3.22873688, -1.57654619],
       [ 1.35561001, -2.81222463, -2.13115191, -2.63512373,  1.24437428,
        -1.79949391,  1.96565115,  0.32127666,  2.91450787, -1.01297271]], dtype=float32), array([[ 1.04543817,  0.25350007, -0.19416428, -2.99677587,  1.09158456,
        -0.77419585,  1.31118512,  0.90980273,  1.60091817,  0.72896349],
       [ 0.44807476, -0.114858  ,  0.4254725 , -1.96435654, -1.16465545,
        -0.60571432, -1.9323138 , -3.09183359, -0.66767651,  1.18132424],
       [ 0.41804138,  0.8005898 ,  1.34982419, -1.11951494, -0.60257638,
        -0.65733844, -0.62484217, -1.05114794, -1.35079217,  1.18004894],
       [-2.24554753, -0.41718578, -0.7750501 ,  2.22614479,  0.74070621,
         0.37609592, -0.85759938, -0.06744213, -1.64424121, -3.28154778],
       [ 0.69541192,  0.49870071, -0.93701386, -0.52575213,  0.53389025,
         0.

9900 0.00133427 [array([[ 1.05357563, -2.55114484, -2.25446987, -2.55373406,  1.68697715,
        -2.64080667,  2.2310245 , -1.29365337,  3.23290515, -1.57704794],
       [ 1.3592931 , -2.81950188, -2.13730431, -2.64482117,  1.24955058,
        -1.80670989,  1.9682318 ,  0.32487234,  2.92035365, -1.01434088]], dtype=float32), array([[ 1.04878545,  0.25424841, -0.19406419, -3.00154352,  1.09258425,
        -0.77455831,  1.31267023,  0.9119572 ,  1.60753787,  0.73377782],
       [ 0.45093936, -0.11504568,  0.42664209, -1.96925402, -1.167045  ,
        -0.60567856, -1.93468869, -3.09454775, -0.67000973,  1.19152153],
       [ 0.41997665,  0.80026287,  1.35082293, -1.12279296, -0.60440892,
        -0.65698969, -0.6268363 , -1.05334938, -1.35393286,  1.18806422],
       [-2.25101209, -0.41914636, -0.77470732,  2.23526573,  0.74278355,
         0.37981561, -0.85751188, -0.06674064, -1.65805793, -3.28849602],
       [ 0.69958121,  0.4997021 , -0.9368574 , -0.53213888,  0.53451699,
         0.